In [11]:
from scipy.stats import mannwhitneyu
import json

In [4]:
accuracies_file = "/workspaces/RAG_secure_code_generation/accuracies.json"

In [5]:
with open(accuracies_file) as f:
    accuracies = json.load(f)

In [9]:
print(*accuracies.values())

[0.5130434782608696, 0.5130434782608696, 0.8364839319470699, 0.9795841209829868, 0.5130434782608696, 0.7525519848771266, 0.8793950850661626, 0.5130434782608696, 0.5374291115311909, 0.5132325141776938, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5071833648393195, 0.8368620037807183, 0.834593572778828, 0.5128544423440453, 0.5130434782608696, 0.834593572778828, 0.5130434782608696, 0.5128544423440453, 0.5130434782608696, 0.5130434782608696, 0.5132325141776938, 0.5285444234404537, 0.6835538752362949, 0.8474480151228734, 0.5132325141776938, 0.8400756143667297, 0.5325141776937619, 0.9775047258979206, 0.5374291115311909, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5071833648393195, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0.5130434782608696, 0

In [12]:
stat = mannwhitneyu(*accuracies.values())

In [13]:
stat

MannwhitneyuResult(statistic=9725650.0, pvalue=0.01553205028797757)